In [1]:
from google.cloud import bigquery
import pandas as pd
import os
import time

os.environ['GOOGLE_APPLICATION+CREDENTIALS'] = '/Users/nrabbani/.config/gcloud/application_default_credentials.json'
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions'
%load_ext google.cloud.bigquery

client=bigquery.Client()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

In [2]:
query = '''
SELECT * FROM njr_labutil.lradt
WHERE
    (pat_class = 'Inpatient' OR pat_class = 'Surgery Admit')
    AND pat_service NOT IN (
        'Obstetrics', 'Maternal-Fetal Medicine',
        'Reproductive Endocrinology and Infertility',
        'Gynecology & Obstetrics', 'Gynecology',
        'Intensive Care', 'Neonatology', 'Cardiovascular Intensive Care')
    AND (pat_lv_of_care = 'Acute Care' OR pat_lv_of_care='')
    AND accomodation IN (
            'PEDS MED/SURG Level 1','PEDS MED/SURG Level 2', 'PEDS MED/SURG Level 3',
            'PEDS Med/Surg',
            'CCP Med/Surg Level 1','CCP Med/Surg Level 2','CCP Med/Surg Level 3',
            'PEDS TXP Med/Surg Level 1', 'PEDS TXP Med/Surg Level 2','PEDS TXP Med/Surg Level 3',
            'Oncology Level 1', 'Oncology Level 2', 'Oncology Level 3', 'Oncology Level 4', 'SCT', 'Onc Med/Surg', 'SCT 2 (SCT 2)',
            'ICN Lvl A', 'ICN Lvl B', 'ICN Lvl C', 'ICN Lvl D'
    )
'''
query_job = client.query(query)

In [3]:
tic = time.time()
lradt = query_job.to_dataframe()
print(time.time() - tic)

834.7488169670105


In [4]:
lradt.loc[~lradt.ord_num_value.isnull()].to_pickle('data/lradt_20211025.pkl')